# 160 : Demo Hierarchical NER with joint-labels

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

2


In [9]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR').resolve()
  OUT_BASE = Path('/work/stual/res_ICDAR/method_3/').resolve()

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m3_hierarchical_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m3_hierarchical_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_3


### Constants

In [10]:
#Model to run
MODEL_BASE = OUT_BASE / 'tmp'
FINETUNED_MODEL = MODEL_BASE / '323-pretrained-camembert-ner-hierarchical-loss-io/checkpoint-1000/'
FINETUNED_MODEL

PosixPath('/work/stual/res_ICDAR/method_3/tmp/323-pretrained-camembert-ner-hierarchical-loss-io/checkpoint-1000')

In [11]:
import csv
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

LIMIT = 100

tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL) #Param : tokenizer du modele souhaité
model = AutoModelForTokenClassification.from_pretrained(FINETUNED_MODEL) #Modèle choisi

#Classification des entités
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy=None, use_fast=True)

### Data

In [12]:
samples = """☞  T Dufant (Victor), libraire, r. du Gros-Che- net. 2. JO \n
Dutay, essayeur du commerce, place Dau-  - píině, 5. DOB \n
Dulay, chandronnier, r. du Pont- aux- Chuux, 15. SI \n
Dufay (V.e), grenetière, r. du Fauh.-S. Denis, 20. 372 \n
Y ☞ Dnten,charentier, 1. Montmartre, 89. 318 \n
Dufey fils, bijoutier, passage de la Réunion A \n
Dnley, boucher, r. S.&quot;Louis-au-Marais,3. * \n
☞ Duffand, bijonteer, r. S.-Martin, 107. 284 \n
Duftaud, maçon, r. S-Honore  353. O \n
Dullaut, chandronnier, r. de la Sourdière I- ( ☞T4 \n
lukils. limonauier, 1. des Colonnes, 6. J9 \n
— T Dullocu, architecte, r. Basse-Porte-S.De- — Ris, 28. \n
Duflot, loueur de carrosses, r. de Paradis- 505 Poissonnière, 22. \n
Dufort, bottier, Palais-R., gal. vitrée, 215. A \n
Dufort fils, bottier (exp. 1819.), r. J.-J.. Rousscan. 18. 290 \n
* Dufort, tabletier, r. Jean-Pain-Mollet, 10. 4O \n
Dufossé, teiniurier, r. du Faub.-Montmar- tre , 63. 483 \n
Duſour el Besnard, march, de bois à bruler, quai de la Tournelle, 17. etr. des Fossés- SBernard. 11. Dí \n
Dufour jeune, march, de bois à brüler de la Pépinière, 53. 7 r. \n
Dufour (Mad.), march. de chapeaux de paitle, passage dn Panorama, 11. IO \n
Dutour, thapelier, r. S.-Honore 383. 3og \n
Dufour, cirier, r. de PArbre-Sec, 6o. 316 \n
Dufour et co., commiss., r. des Contures-  DAO S. Dervajs, 18. \n
Dulour, corropeir, 1. des Vertusy1d 3 \n
Dufour (Charles), épicier, r. S.-Denis, Jo7. ☞ 332 \n
Dufour, fab. d&apos;éventails (erp. 1819) Beaubourg, 48.  360 \n
Dufour, faiencter, r. S.-Anne , 50. 302 \n
Dufour, A. Sarnt,r. Ventadour, ☞7 \n
Dufour (Gabriel), libraire, r. de Vaugirard, A \n
Dufour, maçon, r. S.-Jean-Baptiste, 4 402 \n
Dufour, architecte, r. de Chartres-S.-Ho- 2781 40a nore, 12. (Elig.) \n
Dufour (Mad.), nouveautes, 1. Neuve-des- H12 Petits-Champs, 4. \n
Dufour Chabrol, papetier, r. S.-Martin ☞ 4 \n
Dufour (J.) et co,, fab. de papiers peints, ((1819); r. Beauvean-S. Antoine, 19."""

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/IPython/core/inputtransformer2.py:595: UserWarning: `make_tokens_by_line` received a list of lines which do not have lineending markers ('\n', '\r', '\r\n', '\x0b', '\x0c'), behavior will be unspecified
  tokens_by_line = make_tokens_by_line(lines)


### Exemples

In [14]:
from xmlize_util import get_NER_tags, xmlize_multilevel

for s in samples.split('\n'):
    if s != '':
        print(s)
        res = nlp(s)
        levels, num_levels = get_NER_tags(s,res)
        #print(levels)
        xml = xmlize_multilevel(levels,num_levels)
        
        #Correction IO
        #xml = xml.replace('</LOC></SPAT>.<SPAT><LOC>','')
        #xml = xml.replace('</LOC></SPAT><ACT>.</ACT><SPAT><LOC>','')
        """xml = xml.replace('<SPAT>.</SPAT> ','')"""
        
        #Correction IOB2
        #xml = xml.replace('</PER><PER>','') #> I-PER suivi par un B-PER I-PER B-PER
        #xml = xml.replace('</ACT>&amp;<ACT>','')
        #</LOC></SPAT><PER>.</PER><SPAT><LOC>
        #</ACT></DESC><DESC><ACT>
        #<DESC>.</DESC>

        #Corrections ptrn IO
        #</LOC>.<LOC>
        
        #Confusion TITRE-P et TITRE </DESC><TITRE>(18</TITRE><DESC>19</DESC>
        print(xml)
        print('')

☞  T Dufant (Victor), libraire, r. du Gros-Che- net. 2. JO 
  label  start  end                     span
0     O      0    4                     ☞  T
1   PER      4   21         Dufant (Victor),
2   ACT     21   30                 libraire
3     O     30   31                        ,
4  SPAT     31   54   r. du Gros-Che- net. 2
5     O     54   59                    . JO 
      label  start  end                        span
0         O      0    4                        ☞  T
1         O      4   30   Dufant (Victor), libraire
2         O     30   31                           ,
3       LOC     31   51         r. du Gros-Che- net
4         O     51   52                           .
5  CARDINAL     52   54                           2
6         O     54   59                       . JO 
☞  T<PER> Dufant (Victor),</PER><ACT> libraire</ACT>,<SPAT><LOC> r. du Gros-Che- net</LOC>.<CARDINAL> 2</CARDINAL></SPAT>. JO 

Dutay, essayeur du commerce, place Dau-  - píině, 5. DOB 
  label  start  end    

<PER>lukils</PER>.<ACT> limonauier</ACT>,<SPAT><LOC> 1. des Colonnes</LOC>,<CARDINAL> 6</CARDINAL></SPAT>. J9 

— T Dullocu, architecte, r. Basse-Porte-S.De- — Ris, 28. 
  label  start  end                             span
0     O      0    3                              — T
1   PER      3   11                          Dullocu
2   ACT     11   23                     , architecte
3     O     23   24                                ,
4  SPAT     24   55   r. Basse-Porte-S.De- — Ris, 28
5     O     55   57                               . 
      label  start  end                         span
0         O      0    3                          — T
1         O      3   23          Dullocu, architecte
2         O     23   24                            ,
3       LOC     24   51   r. Basse-Porte-S.De- — Ris
4         O     51   52                            ,
5  CARDINAL     52   55                           28
6         O     55   57                           . 
— T<PER> Dullocu</PER><ACT>, archit

  label  start  end                span
0   PER      0    6              Dutour
1   ACT      6   17         , thapelier
2     O     17   18                   ,
3  SPAT     18   36   r. S.-Honore 383.
4     O     36   41                3og 
      label  start  end               span
0         O      0   17  Dutour, thapelier
1         O     17   18                  ,
2       LOC     18   31       r. S.-Honore
3  CARDINAL     31   36               383.
4         O     36   41               3og 
<PER>Dutour</PER><ACT>, thapelier</ACT>,<SPAT><LOC> r. S.-Honore</LOC><CARDINAL> 383.</CARDINAL></SPAT> 3og 

Dufour, cirier, r. de PArbre-Sec, 6o. 316 
  label  start  end                   span
0   PER      0    6                 Dufour
1   ACT      6   14               , cirier
2     O     14   15                      ,
3  SPAT     15   36   r. de PArbre-Sec, 6o
4     O     36   42                 . 316 
      label  start  end               span
0         O      0   14     Dufour, cirier
1    

  label  start  end                                 span
0   PER      0   14                       Dufour (Mad.),
1   ACT     14   25                           nouveautes
2     O     25   26                                    ,
3  SPAT     26   61   1. Neuve-des- H12 Petits-Champs, 4
4     O     61   63                                   . 
      label  start  end                              span
0         O      0   25         Dufour (Mad.), nouveautes
1         O     25   26                                 ,
2       LOC     26   58   1. Neuve-des- H12 Petits-Champs
3         O     58   59                                 ,
4  CARDINAL     59   61                                 4
5         O     61   63                                . 
<PER>Dufour (Mad.),</PER><ACT> nouveautes</ACT>,<SPAT><LOC> 1. Neuve-des- H12 Petits-Champs</LOC>,<CARDINAL> 4</CARDINAL></SPAT>. 

Dufour Chabrol, papetier, r. S.-Martin ☞ 4 
  label  start  end               span
0   PER      0   14     Dufour Chabro